In [2]:
import sys

sys.path.append("../../")

from abc import ABC, abstractmethod
from typing import Any, AsyncGenerator, Union
import os
from langchain_aws import ChatBedrock


class BaseChatbot(ABC):
    @abstractmethod
    def get_text_response(self, prompt: str) -> str:
        pass

    @abstractmethod
    async def get_text_response_async(self, prompt: str) -> Union[str, list[Union[str, dict[Any, Any]]]]:
        pass

    @abstractmethod
    async def stream_response(self, prompt: str) -> AsyncGenerator[Union[str, list[Union[str, dict[Any, Any]]]], None]:
        if False:
            yield


class ClaudeHaikuChatbot(BaseChatbot):
    def __init__(self, temperature: float = 0):
        self.llm = ChatBedrock(
            model="anthropic.claude-3-haiku-20240307-v1:0",
            model_kwargs={"temperature": temperature},
            credentials_profile_name=os.getenv("AWS_PROFILE"),
            region=os.getenv("AWS_REGION", "us-east-1"),
        )

    def get_text_response(self, prompt: str) -> str:
        response = self.llm.invoke(prompt)
        return response.content

    async def get_text_response_async(self, prompt: str) -> Union[str, list[Union[str, dict[Any, Any]]]]:
        response = await self.llm.ainvoke(prompt)
        return response.content

    async def stream_response(self, prompt: str) -> AsyncGenerator[Union[str, list[Union[str, dict[Any, Any]]]], None]:
        async for chunk in self.llm.astream(prompt):
            yield chunk.content


# Test Haiku
haiku_chatbot = ClaudeHaikuChatbot(temperature=0.7)
print("Haiku streaming response:")
async for chunk in haiku_chatbot.stream_response("Tell me a short joke"):
    print(chunk, end="")
print()

Haiku streaming response:
Here's a short joke for you:

Why don't scientists trust atoms? Because they make up everything!


In [3]:
class ClaudeSonnetChatbot(BaseChatbot):
    def __init__(self, temperature: float = 0):
        self.llm = ChatBedrock(
            model="anthropic.claude-3-sonnet-20240229-v1:0",
            model_kwargs={"temperature": temperature},
            credentials_profile_name=os.getenv("AWS_PROFILE"),
            region=os.getenv("AWS_REGION", "us-east-1"),
        )

    def get_text_response(self, prompt: str) -> str:
        response = self.llm.invoke(prompt)
        return response.text()

    async def get_text_response_async(self, prompt: str) -> Union[str, list[Union[str, dict[Any, Any]]]]:
        response = await self.llm.ainvoke(prompt)
        return response.content

    async def stream_response(self, prompt: str) -> AsyncGenerator[Union[str, list[Union[str, dict[Any, Any]]]], None]:
        async for chunk in self.llm.astream(prompt):
            yield chunk.content


# Test Claude 3 Sonnet (supports on-demand throughput)
sonnet_chatbot = ClaudeSonnetChatbot(temperature=0.7)
text_response = sonnet_chatbot.get_text_response("What is the meaning of life?")
print("Claude 3 Sonnet text response:", text_response)

print("Claude 3 Sonnet streaming response:")
async for chunk in sonnet_chatbot.stream_response("Explain quantum computing in simple terms"):
    print(chunk, end="")
print()

Claude 3 Sonnet text response: There is no single agreed upon answer to the meaning of life, as it is a deep philosophical question that people have grappled with for centuries. Some of the major perspectives on it include:

- Religious/spiritual views that propose life's meaning comes from a higher power or divine purpose.

- Existentialist perspectives that we have to create our own subjective meaning to life.

- Humanist views that meaning arises from human relationships, pursuits and making the world better.

- Nihilist views that life has no inherent meaning.

- Scientific views that life's meaning lies in our evolutionary purpose of reproducing and perpetuating the species.

Ultimately, the meaning of life is something each individual has to grapple with and decide what makes sense to them based on their beliefs, experiences and principles. There is no objectively provable answer that applies universally. The search for life's meaning is one of the deepest driving forces and grea